# Búsqueda Informada

In [1]:
import problema_espacio_estados as probee
import busqueda_espacio_estados as busqee
import copy

## Rompecabezas de las Torres de Hanoi con heurística

El rompecabezas de las Torres de Hanoi consta de tres varillas verticales y un número de discos, que determinará la complejidad del problema, todos de distinto tamaño y apilados de mayor a menor radio en la primera varilla.

El objetivo del juego es pasar todos los discos de la primera a la última varilla, siguiendo tres simples reglas:
1. Se desplaza un disco cada vez.
2. Solo se pueden desplazar los discos de arriba de las varillas.
3. No se puede colocar un disco sobre otro más pequeño.

En esta primera parte de la práctica se mostrará cómo implementar el rompecabezas de las Torres de Hanoi como un problema de espacio de estados y se aplicarán distintos algoritmos de búsqueda para resolverlo.

El primer paso es decidir cómo se van a implementar los estados. Para el rompecabezas de las Torres de Hanoi una opción es hacerlo mediante una lista que guarde para cada varilla el conjunto de los discos que hay en ella.

In [2]:
estado1 = [{2}, set(), {1}]
estado2 = [{1}, set(), {2}]

A continuación hay que implementar las acciones como instancias de la clase `Acción`, proporcionando un nombre, una función de aplicabilidad y una función de aplicación para cada acción. Por ejemplo, la acción `De 1 a 3` que mueve un disco de la primera a la tercera varilla se puede implementar de la siguiente manera:

In [3]:
def esta_vacia(estado, varilla):
    return not estado[varilla - 1]

def disco_superior(estado, varilla):
    return min(estado[varilla - 1])

def aplicabilidad(estado):
    return (not esta_vacia(estado, 1) and
            (esta_vacia(estado, 3) or
             disco_superior(estado, 1) < disco_superior(estado, 3)))

def quitar_disco(estado, varilla):
    disco = disco_superior(estado, varilla)
    estado[varilla - 1].remove(disco)
    return disco

def poner_disco(estado, varilla, disco):
    estado[varilla - 1].add(disco)

def aplicacion(estado):
    nuevo_estado = copy.deepcopy(estado)
    disco = quitar_disco(nuevo_estado, 1)
    poner_disco(nuevo_estado, 3, disco)
    return nuevo_estado

a13 = probee.Accion('De 1 a 3', aplicabilidad, aplicacion)

In [4]:
a13.es_aplicable(estado1)

False

In [5]:
a13.es_aplicable(estado2)

True

In [6]:
a13.aplicar(estado2)

[set(), set(), {1, 2}]

Normalmente las acciones se pueden agrupar en distintos tipos, cada uno de los cuales puede ser implementado de manera abstracta mediante una clase que herede de la clase `Accion`.

Para el rompecabezas de las Torres de Hanoi, todas las acciones son del tipo mover un disco de una varilla a otra. En este caso, consideramos que el coste de mover un disco es siempre 1, el valor por defecto. En caso de que fuera distinto, al crear una instancia de la clase `Accion` se puede proporcionar una función `coste`, o bien al heredar de la clase `Accion` se puede redefinir el método `coste_de_aplicar`.

In [7]:
class MoverDisco(probee.Accion):
    def __init__(self, i, j):
        nombre = 'De {} a {}'.format(i, j)
        super().__init__(nombre)
        self.varilla_de = i
        self.varilla_a = j
    
    def esta_vacia(self, estado, varilla):
        return not estado[varilla - 1]
    
    def disco_superior(self, estado, varilla):
        return min(estado[varilla - 1])
    
    def es_aplicable(self, estado):
        return (not self.esta_vacia(estado, self.varilla_de) and
                (self.esta_vacia(estado, self.varilla_a) or
                 self.disco_superior(estado, self.varilla_de) < 
                 self.disco_superior(estado, self.varilla_a)))
    
    def quitar_disco(self, estado, varilla):
        disco = self.disco_superior(estado, varilla)
        estado[varilla - 1].remove(disco)
        return disco

    def poner_disco(self, estado, varilla, disco):
        estado[varilla - 1].add(disco)
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        disco = self.quitar_disco(nuevo_estado, self.varilla_de)
        self.poner_disco(nuevo_estado, self.varilla_a, disco)
        return nuevo_estado

Finalmente, un problema de espacio de estados se implementa como una instancia de la clase `ProblemaEspacioEstados`, proporcionando una lista de acciones, un estado inicial y una lista de estados finales.

In [8]:
acciones = [MoverDisco(i, j) for i in range(1, 4) for j in range(1, 4) if i != j]
estado_inicial = [{1, 2}, set(), set()]
estado_final = [set(), set(), {1, 2}]
Torres_Hanoi_2_discos = probee.ProblemaEspacioEstados(
    acciones, estado_inicial, [estado_final])

In [9]:
Torres_Hanoi_2_discos.es_estado_final(estado1)

False

In [10]:
Torres_Hanoi_2_discos.es_estado_final(a13.aplicar(estado2))

True

In [11]:
for accion in Torres_Hanoi_2_discos.acciones_aplicables(estado1):
    print(accion.nombre)

De 1 a 2
De 3 a 1
De 3 a 2


In [12]:
for accion in Torres_Hanoi_2_discos.acciones_aplicables(estado1):
    print(accion.aplicar(estado1))

[set(), {2}, {1}]
[{1, 2}, set(), set()]
[{2}, {1}, set()]


El procedimiento para realizar una búsqueda en un espacio de estados consiste en crear una instancia de una clase que implemente un algoritmo de búsqueda, proporcionando los argumentos necesarios, y aplicar el método buscar de esa instancia al problema de espacio de estados.

Las clases correspondientes a los algoritmos de búsqueda más comunes son las siguientes:
* `BusquedaEnAnchura`
* `BusquedaEnProfundidad`
* `BusquedaPrimeroElMejor`: hay que proporcionar la función de evaluación heurística `f`.
* `BusquedaOptima`
* `BusquedaAEstrella`: hay que proporcionar la función de estimación del coste `h`.

Adicionalmente, todas las clases anteriores admiten establecer el argumento `detallado` a `True`, para que al realizar una búsqueda se imprima por pantalla su traza.

In [13]:
b_anchura = busqee.BusquedaEnAnchura(detallado=True)

In [14]:
b_anchura.buscar(Torres_Hanoi_2_discos)

Nodo: Estado: [{1, 2}, set(), set()]; Prof: 0
  Nodo: Estado: [{2}, {1}, set()]; Prof: 1
  Nodo: Estado: [{2}, set(), {1}]; Prof: 1
    Nodo: Estado: [set(), {1}, {2}]; Prof: 2
    Nodo: Estado: [set(), {2}, {1}]; Prof: 2
      Nodo: Estado: [{1}, set(), {2}]; Prof: 3
      Nodo: Estado: [set(), set(), {1, 2}]; Prof: 3


['De 1 a 2', 'De 1 a 3', 'De 2 a 3']

In [15]:
b_profundidad = busqee.BusquedaEnProfundidad(detallado=True)

In [16]:
b_profundidad.buscar(Torres_Hanoi_2_discos)

Nodo: Estado: [{1, 2}, set(), set()]; Prof: 0
  Nodo: Estado: [{2}, set(), {1}]; Prof: 1
    Nodo: Estado: [set(), {2}, {1}]; Prof: 2
      Nodo: Estado: [set(), {1, 2}, set()]; Prof: 3
      Nodo: Estado: [{1}, {2}, set()]; Prof: 3
        Nodo: Estado: [{1}, set(), {2}]; Prof: 4
          Nodo: Estado: [set(), set(), {1, 2}]; Prof: 5


['De 1 a 3', 'De 1 a 2', 'De 3 a 1', 'De 2 a 3', 'De 1 a 3']

Podemos parametrizar la implementación del rompecabezas de las Torres de Hanoi para que dependa del número `n` de discos. Para ello basta implementar una clase que herede de la clase `ProblemaEspacioEstados`. Aprovechamos también para, en lugar de enumerar los estados finales, realizar una descripción declarativa de los mismos redefiniendo el método `es_estado_final`.

In [17]:
class TorresHanoi(probee.ProblemaEspacioEstados):
    def __init__(self, n):
        acciones = [MoverDisco(i, j) for i in range(1, 4) for j in range(1, 4) if i != j]
        estado_inicial = [set(range(1, n + 1)), set(), set()]
        super().__init__(acciones, estado_inicial)
        self.n = n
    
    def es_estado_final(self, estado):
        return estado[2] == set(range(1, self.n + 1))

Con un número de discos igual a 8, el coste en tiempo de los algoritmos de búsqueda en anchura y profundidad comienza a no ser asumible, por lo que debemos pasar a realizar una búsqueda informada.

In [18]:
Torres_Hanoi_8_discos = TorresHanoi(8)

In [19]:
b_anchura = busqee.BusquedaEnAnchura()

In [20]:
from time import time
ti = time()
sol = b_anchura.buscar(Torres_Hanoi_8_discos)
print('Tiempo transcurrido: {} segundos.'.format(round(time() - ti,2)))

Tiempo transcurrido: 6.65 segundos.


Para poder aplicar la búsqueda $A^*$, es un requisito necesario definir una función que para cada nodo estime el coste de una solución óptima desde el estado de ese nodo (que en nuestra implementación está guardado en el atributo `estado` de la clase que implementa a estos últimos).

#### Definir una función heurística para el rompecabezas de las Torres de Hanoi

In [ ]:
def h(estado):
    #TO-DO
    return len(estado[0]) + len(estado[1])
b_a_estrella = busqee.BusquedaAEstrella(h)

In [22]:
from time import time
ti = time()
sol = b_a_estrella.buscar(Torres_Hanoi_8_discos)
print('Tiempo transcurrido: {} segundos.'.format(round(time() - ti,2)))

Tiempo transcurrido: 5.43 segundos.


#### Evaluando el tiempo de ejecución para diferente número de discos

In [23]:
from time import time
import pandas as pd

def medir_tiempo_ejecuciones(lista_N_discos, algoritmos, detallado=True):
    df_tiempos = pd.DataFrame(columns=algoritmos.keys())
    for n_discos in lista_N_discos:
        Torres_Hanoi_N_discos = TorresHanoi(n_discos)
        fila_resultados = {'N_Discos': n_discos}
        for nombre_alg, alg in algoritmos.items():
            ti = time()
            sol = alg.buscar(Torres_Hanoi_N_discos)
            tf = time() - ti
            fila_resultados[nombre_alg] = tf
            df_tiempos.append(fila_resultados,ignore_index=True)
            if detallado:
                print('N_Torres: {} Alg: {} T: {} Pasos_sol: {}'.format(n_discos, nombre_alg, tf, len(sol)))
    return df_tiempos   

Recordemos que como todas las acciones tienen el mismo coste, tanto la búsqueda en anchura como la búsqueda óptima proporcionan una solución de mínimo coste. Esto se debe a que, dado que todas las acciones tienen el mismo coste, la solución de mínimo coste es aquella con el menor número de pasos.

In [24]:
algoritmos_min = {'Anchura': busqee.BusquedaEnAnchura(),  
              'Estrella': busqee.BusquedaAEstrella(h)}

In [25]:
res = medir_tiempo_ejecuciones(range(2,9), algoritmos_min)

N_Torres: 2 Alg: Anchura T: 0.0003581047058105469 Pasos_sol: 3
N_Torres: 2 Alg: Estrella T: 0.0003871917724609375 Pasos_sol: 3
N_Torres: 3 Alg: Anchura T: 0.0015485286712646484 Pasos_sol: 7
N_Torres: 3 Alg: Estrella T: 0.0012798309326171875 Pasos_sol: 7
N_Torres: 4 Alg: Anchura T: 0.004664182662963867 Pasos_sol: 15
N_Torres: 4 Alg: Estrella T: 0.004266977310180664 Pasos_sol: 15
N_Torres: 5 Alg: Anchura T: 0.022788524627685547 Pasos_sol: 31
N_Torres: 5 Alg: Estrella T: 0.01798272132873535 Pasos_sol: 31
N_Torres: 6 Alg: Anchura T: 0.11616063117980957 Pasos_sol: 63
N_Torres: 6 Alg: Estrella T: 0.10486674308776855 Pasos_sol: 63
N_Torres: 7 Alg: Anchura T: 0.8500230312347412 Pasos_sol: 127
N_Torres: 7 Alg: Estrella T: 0.7170979976654053 Pasos_sol: 127
N_Torres: 8 Alg: Anchura T: 6.122084856033325 Pasos_sol: 255
N_Torres: 8 Alg: Estrella T: 5.744111776351929 Pasos_sol: 255


### De los resultados obtenidos sacamos dos conclusiones
- **Heurística:** Nuestra heurística no aporta gran cosa. Dada la naturaleza del problema, no es fácil dar con una buena heurística.
- **Complejidad temporal:** Exponencial.

# 8-puzzle

Ahora vamos a definir las acciones necesarias para resolver el problema 8-Puzzle visto en clase.
- Existen varias representaciones posibles para el estado del puzzle. Las dos más intuitivas son:
    - Vector: [1, 2, 3, 8, 0, 4, 7, 6, 5]
    - Matriz: [[1, 2, 3], [8, 0, 4], [7, 6, 5]]
- En la siguiente implementación se ha usado la primera.
- El cero denota el hueco, también podría haberse usado la letra 'H', por ejemplo.

**Nota:** Al final del notebook se repite la implementación del problema del 8-puzzle usando la segúnda representación, es decir, una lista de listas (matriz).

In [26]:
class MoverArriba(probee.Accion):
    def __init__(self):
        super().__init__("Mover hueco hacia arriba")
    
    def es_aplicable(self, estado):
        pos_hueco = estado.index(0)
        return pos_hueco not in [0, 1, 2]
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        pos_hueco = estado.index(0)
        nueva_pos = pos_hueco - 3
        nuevo_estado[pos_hueco], nuevo_estado[nueva_pos] = nuevo_estado[nueva_pos], nuevo_estado[pos_hueco] # Intercambiamos las posiciones
        return nuevo_estado 

    
class MoverAbajo(probee.Accion):
    def __init__(self):
        super().__init__("Mover hueco hacia abajo")
    
    def es_aplicable(self, estado):
        pos_hueco = estado.index(0)
        return pos_hueco not in [6,7,8]
        
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        pos_hueco = estado.index(0)
        nueva_pos = pos_hueco + 3
        nuevo_estado[pos_hueco], nuevo_estado[nueva_pos] = nuevo_estado[nueva_pos], nuevo_estado[pos_hueco]
        return nuevo_estado


class MoverDerecha(probee.Accion):
    def __init__(self):
        super().__init__("Mover hueco hacia la derecha")
    
    def es_aplicable(self, estado):
        pos_hueco = estado.index(0)
        return pos_hueco not in [2,5,8]
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        pos_hueco = estado.index(0)
        nueva_pos = pos_hueco + 1
        nuevo_estado[pos_hueco], nuevo_estado[nueva_pos] = nuevo_estado[nueva_pos], nuevo_estado[pos_hueco]
        return nuevo_estado
    

class MoverIzquierda(probee.Accion):
    def __init__(self):
        super().__init__("Mover hueco hacia la izquierda")
    
    def es_aplicable(self, estado):
        pos_hueco = estado.index(0)
        return pos_hueco not in [0,3,6]
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        pos_hueco = estado.index(0)
        nueva_pos = pos_hueco - 1
        nuevo_estado[pos_hueco], nuevo_estado[nueva_pos] = nuevo_estado[nueva_pos], nuevo_estado[pos_hueco]
        return nuevo_estado

In [27]:
from time import time

acciones = [MoverArriba(), MoverAbajo(), MoverDerecha(), MoverIzquierda()]
estado_final = [1, 2, 3, 8, 0, 4, 7, 6, 5]
estado_inicial_1 = [2, 8, 3, 1, 6, 4, 7, 0, 5]
estado_inicial_2 = [4, 8, 1, 3, 0, 2, 7, 6, 5]
estado_inicial_3 = [2, 1, 6, 4, 0, 8, 7, 5, 3]
estado_inicial_4 = [5, 2, 3, 0, 4, 8, 7, 6, 1]
puzzle_8_1 = probee.ProblemaEspacioEstados(acciones, estado_inicial_1, [estado_final])
puzzle_8_2 = probee.ProblemaEspacioEstados(acciones, estado_inicial_2, [estado_final])
puzzle_8_3 = probee.ProblemaEspacioEstados(acciones, estado_inicial_3, [estado_final])
puzzle_8_4 = probee.ProblemaEspacioEstados(acciones, estado_inicial_4, [estado_final])

¿Necesitamos una heurística?

In [28]:
ti = time()
b_anchura = busqee.BusquedaEnAnchura(detallado=True)
sol1 = b_anchura.buscar(puzzle_8_1)
print('Tiempo transcurrido: {} segundos. Tamaño solución: {}'.format(round(time() - ti,2), len(sol1)))
sol1

Nodo: Estado: [2, 8, 3, 1, 6, 4, 7, 0, 5]; Prof: 0
  Nodo: Estado: [2, 8, 3, 1, 0, 4, 7, 6, 5]; Prof: 1
  Nodo: Estado: [2, 8, 3, 1, 6, 4, 7, 5, 0]; Prof: 1
  Nodo: Estado: [2, 8, 3, 1, 6, 4, 0, 7, 5]; Prof: 1
    Nodo: Estado: [2, 0, 3, 1, 8, 4, 7, 6, 5]; Prof: 2
    Nodo: Estado: [2, 8, 3, 1, 4, 0, 7, 6, 5]; Prof: 2
    Nodo: Estado: [2, 8, 3, 0, 1, 4, 7, 6, 5]; Prof: 2
    Nodo: Estado: [2, 8, 3, 1, 6, 0, 7, 5, 4]; Prof: 2
    Nodo: Estado: [2, 8, 3, 0, 6, 4, 1, 7, 5]; Prof: 2
      Nodo: Estado: [2, 3, 0, 1, 8, 4, 7, 6, 5]; Prof: 3
      Nodo: Estado: [0, 2, 3, 1, 8, 4, 7, 6, 5]; Prof: 3
      Nodo: Estado: [2, 8, 0, 1, 4, 3, 7, 6, 5]; Prof: 3
      Nodo: Estado: [2, 8, 3, 1, 4, 5, 7, 6, 0]; Prof: 3
      Nodo: Estado: [0, 8, 3, 2, 1, 4, 7, 6, 5]; Prof: 3
      Nodo: Estado: [2, 8, 3, 7, 1, 4, 0, 6, 5]; Prof: 3
      Nodo: Estado: [2, 8, 0, 1, 6, 3, 7, 5, 4]; Prof: 3
      Nodo: Estado: [2, 8, 3, 1, 0, 6, 7, 5, 4]; Prof: 3
      Nodo: Estado: [0, 8, 3, 2, 6, 4, 1, 7, 5]; Prof: 3
  

['Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

In [29]:
# ti = time()
# b_profundidad = busqee.BusquedaEnProfundidad(detallado=False)
# sol = b_profundidad.buscar(puzzle_8_1)
# print('Tiempo transcurrido: {} segundos. Tamaño solución: {}'.format(round(time() - ti,2), len(sol)))
# sol

In [30]:
ti = time()
b_anchura = busqee.BusquedaEnAnchura(detallado=True)
sol2 = b_anchura.buscar(puzzle_8_2)
print('Tiempo transcurrido: {} segundos. Tamaño solución: {}'.format(round(time() - ti,2), len(sol2)))
sol2

Nodo: Estado: [4, 8, 1, 3, 0, 2, 7, 6, 5]; Prof: 0
  Nodo: Estado: [4, 0, 1, 3, 8, 2, 7, 6, 5]; Prof: 1
  Nodo: Estado: [4, 8, 1, 3, 6, 2, 7, 0, 5]; Prof: 1
  Nodo: Estado: [4, 8, 1, 3, 2, 0, 7, 6, 5]; Prof: 1
  Nodo: Estado: [4, 8, 1, 0, 3, 2, 7, 6, 5]; Prof: 1
    Nodo: Estado: [4, 1, 0, 3, 8, 2, 7, 6, 5]; Prof: 2
    Nodo: Estado: [0, 4, 1, 3, 8, 2, 7, 6, 5]; Prof: 2
    Nodo: Estado: [4, 8, 1, 3, 6, 2, 7, 5, 0]; Prof: 2
    Nodo: Estado: [4, 8, 1, 3, 6, 2, 0, 7, 5]; Prof: 2
    Nodo: Estado: [4, 8, 0, 3, 2, 1, 7, 6, 5]; Prof: 2
    Nodo: Estado: [4, 8, 1, 3, 2, 5, 7, 6, 0]; Prof: 2
    Nodo: Estado: [0, 8, 1, 4, 3, 2, 7, 6, 5]; Prof: 2
    Nodo: Estado: [4, 8, 1, 7, 3, 2, 0, 6, 5]; Prof: 2
      Nodo: Estado: [4, 1, 2, 3, 8, 0, 7, 6, 5]; Prof: 3
      Nodo: Estado: [3, 4, 1, 0, 8, 2, 7, 6, 5]; Prof: 3
      Nodo: Estado: [4, 8, 1, 3, 6, 0, 7, 5, 2]; Prof: 3
      Nodo: Estado: [4, 8, 1, 0, 6, 2, 3, 7, 5]; Prof: 3
      Nodo: Estado: [4, 0, 8, 3, 2, 1, 7, 6, 5]; Prof: 3
      Nodo: 

['Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

In [31]:
ti = time()
b_anchura = busqee.BusquedaEnAnchura(detallado=False)
sol3 = b_anchura.buscar(puzzle_8_3)
print('Tiempo transcurrido: {} segundos. Tamaño solución: {}'.format(round(time() - ti,2), len(sol3)))
sol3

Tiempo transcurrido: 117.58 segundos. Tamaño solución: 18


['Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

Parece que si. En problemas como el 8-puzzle el estado inicial es muy importante. Vamos a probar diferentes heurísticas.

**Heurística:** número de piezas descolocadas respecto de su posición en el estado final.

In [32]:
def h1_ocho_puzzle(estado):
    # TO-DO
    cont=0
    for x,y in zip(estado,(1,2,3,8,0,4,7,6,5)):
        if x !=0 and x!=y: cont += 1
    return cont

**Heurística:** suma de las distancias Manhattan de cada pieza a donde debería estar en el estado final.

In [33]:
def h2_ocho_puzzle(estado):
    # TO-DO
    posiciones_final=(4,0,1,2,5,8,7,6,3) # posiciones_final[i] es la posición
                                         # de i en el estado final  
    suma = 0                                                            
    for i in range(9):
        estadoi=estado[i]  # valor o casilla que está en la posición i de estado
        if estadoi != 0: # Excluimos el valor cero o 'hueco'.
            j=posiciones_final[estadoi] # j es la posición donde debería estar lael valor o casilla
                                        # que está en la posición i de estado
            i_x,i_y=divmod(i,3) # división entera y resto.
            j_x,j_y=divmod(j,3)
            suma += abs(i_x-j_x)+abs(i_y-j_y) # distancia Manhattan
    return suma

**Heurística no admisible:** La siguiente heurística h3_ocho_puzzle se obtiene sumando a la heurística h2_ocho_puzzle una componente que cuantifica la "secuencialidad" en las casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en A*, con cada uno de los estados anteriores. Comentar los resultados.

In [34]:
def h3_ocho_puzzle(estado):
    # Para cada indice o posición de nuestro estado, este diccionario indica cuál es el siguiente 
    # indice a consultar para seguir el recorrido de las agujas del reloj en el cuadrado.
    suc_ocho_puzzle ={0: 1, 1: 2, 2: 5, 3: 0, 4: 4, 5: 8, 6: 3, 7: 6, 8: 7}  
    
    # Comprueba si hay secuencialidad entre el valor en posición la posición i y el valor en 
    # la siguiente posición (segun las agujas del reloj). Si no hay secuencialidad se 
    # devuelve una penalización de dos unidades.
    def secuencialidad_aux(estado,i):
        val=estado[i]
        if val == 0:
            return 0
        elif i == 4:
            return 1
        else:
            i_sig=suc_ocho_puzzle[i]
            val_sig = (val+1 if val<8 else 1)
            return 0 if val_sig == estado[i_sig] else 2 
    
    # Se comprueba la secuencilidad para todas las posiciones del estado y se suman 
    # las penalizaciones. Finalmente, se emite como resultado una combinación entre la 
    # heurística 2 y las penalizaciones de secuencialidad.
    def secuencialidad(estado):
        res= 0 
        for i in range(8): 
            res+=secuencialidad_aux(estado,i)
        return res    

    return h2_ocho_puzzle(estado) + 3*secuencialidad(estado)

Veamos la valoración que dan las diferentes heurísticas a los 4 estados iniciales que hemos definido.

In [35]:
estados_iniciales = [estado_inicial_1, estado_inicial_2, estado_inicial_3, estado_inicial_4]
for e in estados_iniciales:
    print('Estado: {}. h1: {}. h2: {}. h3: {}.'.
          format(e, h1_ocho_puzzle(e), h2_ocho_puzzle(e), h3_ocho_puzzle(e)))

Estado: [2, 8, 3, 1, 6, 4, 7, 0, 5]. h1: 4. h2: 5. h3: 26.
Estado: [4, 8, 1, 3, 0, 2, 7, 6, 5]. h1: 5. h2: 12. h3: 30.
Estado: [2, 1, 6, 4, 0, 8, 7, 5, 3]. h1: 7. h2: 12. h3: 54.
Estado: [5, 2, 3, 0, 4, 8, 7, 6, 1]. h1: 4. h2: 11. h3: 32.


**Valoración nodos:** Hemos visto que las implementaciones proporcionadas de los diferentes algoritmos de búsqueda, tienen un parámetro `detallado` que permite imprimir información sobre los nodos que el algoritmo va visitando. Para algunos de los algoritmos se proporciona el campo `Valoración` el cual tendrá una interpretación diferente según cada algoritmo. Recordemos, que algunos algoritmos usan una valoración de cada nodo para ordenar la frontera, siendo esta:
- Búsqueda óptima: `coste`
- Búsqueda primero el mejor: `heurística`
- Búsqueda A*: `coste + heurística`

#### Estado inicial 1:

In [36]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h1_ocho_puzzle, detallado=True)
sol = busqueda_aestrella.buscar(puzzle_8_1)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Nodo: Estado: [2, 8, 3, 1, 6, 4, 7, 0, 5]; Prof: 0; Valoración: 4; Coste: 0
  Nodo: Estado: [2, 8, 3, 1, 0, 4, 7, 6, 5]; Prof: 1; Valoración: 4; Coste: 1
    Nodo: Estado: [2, 0, 3, 1, 8, 4, 7, 6, 5]; Prof: 2; Valoración: 5; Coste: 2
    Nodo: Estado: [2, 8, 3, 0, 1, 4, 7, 6, 5]; Prof: 2; Valoración: 5; Coste: 2
      Nodo: Estado: [0, 2, 3, 1, 8, 4, 7, 6, 5]; Prof: 3; Valoración: 5; Coste: 3
        Nodo: Estado: [1, 2, 3, 0, 8, 4, 7, 6, 5]; Prof: 4; Valoración: 5; Coste: 4
          Nodo: Estado: [1, 2, 3, 8, 0, 4, 7, 6, 5]; Prof: 5; Valoración: 5; Coste: 5
Tiempo transcurrido: 0.0020232200622558594. Longitud solución: 5


['Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

In [37]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h2_ocho_puzzle, detallado=True)
sol = busqueda_aestrella.buscar(puzzle_8_1)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Nodo: Estado: [2, 8, 3, 1, 6, 4, 7, 0, 5]; Prof: 0; Valoración: 5; Coste: 0
  Nodo: Estado: [2, 8, 3, 1, 0, 4, 7, 6, 5]; Prof: 1; Valoración: 5; Coste: 1
    Nodo: Estado: [2, 0, 3, 1, 8, 4, 7, 6, 5]; Prof: 2; Valoración: 5; Coste: 2
      Nodo: Estado: [0, 2, 3, 1, 8, 4, 7, 6, 5]; Prof: 3; Valoración: 5; Coste: 3
        Nodo: Estado: [1, 2, 3, 0, 8, 4, 7, 6, 5]; Prof: 4; Valoración: 5; Coste: 4
          Nodo: Estado: [1, 2, 3, 8, 0, 4, 7, 6, 5]; Prof: 5; Valoración: 5; Coste: 5
Tiempo transcurrido: 0.0015645027160644531. Longitud solución: 5


['Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

In [38]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h3_ocho_puzzle, detallado=True)
sol = busqueda_aestrella.buscar(puzzle_8_1)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Nodo: Estado: [2, 8, 3, 1, 6, 4, 7, 0, 5]; Prof: 0; Valoración: 26; Coste: 0
  Nodo: Estado: [2, 8, 3, 1, 0, 4, 7, 6, 5]; Prof: 1; Valoración: 23; Coste: 1
    Nodo: Estado: [2, 0, 3, 1, 8, 4, 7, 6, 5]; Prof: 2; Valoración: 20; Coste: 2
      Nodo: Estado: [0, 2, 3, 1, 8, 4, 7, 6, 5]; Prof: 3; Valoración: 20; Coste: 3
        Nodo: Estado: [1, 2, 3, 0, 8, 4, 7, 6, 5]; Prof: 4; Valoración: 14; Coste: 4
          Nodo: Estado: [1, 2, 3, 8, 0, 4, 7, 6, 5]; Prof: 5; Valoración: 5; Coste: 5
Tiempo transcurrido: 0.0013096332550048828. Longitud solución: 5


['Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

#### Estado inicial 2:

In [39]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h1_ocho_puzzle, detallado=True)
sol = busqueda_aestrella.buscar(puzzle_8_2)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Nodo: Estado: [4, 8, 1, 3, 0, 2, 7, 6, 5]; Prof: 0; Valoración: 5; Coste: 0
  Nodo: Estado: [4, 0, 1, 3, 8, 2, 7, 6, 5]; Prof: 1; Valoración: 6; Coste: 1
  Nodo: Estado: [4, 8, 1, 3, 2, 0, 7, 6, 5]; Prof: 1; Valoración: 6; Coste: 1
  Nodo: Estado: [4, 8, 1, 0, 3, 2, 7, 6, 5]; Prof: 1; Valoración: 6; Coste: 1
  Nodo: Estado: [4, 8, 1, 3, 6, 2, 7, 0, 5]; Prof: 1; Valoración: 7; Coste: 1
    Nodo: Estado: [4, 1, 0, 3, 8, 2, 7, 6, 5]; Prof: 2; Valoración: 7; Coste: 2
    Nodo: Estado: [0, 4, 1, 3, 8, 2, 7, 6, 5]; Prof: 2; Valoración: 7; Coste: 2
    Nodo: Estado: [4, 8, 0, 3, 2, 1, 7, 6, 5]; Prof: 2; Valoración: 7; Coste: 2
    Nodo: Estado: [0, 8, 1, 4, 3, 2, 7, 6, 5]; Prof: 2; Valoración: 7; Coste: 2
    Nodo: Estado: [4, 8, 1, 3, 2, 5, 7, 6, 0]; Prof: 2; Valoración: 8; Coste: 2
    Nodo: Estado: [4, 8, 1, 7, 3, 2, 0, 6, 5]; Prof: 2; Valoración: 8; Coste: 2
      Nodo: Estado: [4, 1, 2, 3, 8, 0, 7, 6, 5]; Prof: 3; Valoración: 8; Coste: 3
      Nodo: Estado: [3, 4, 1, 0, 8, 2, 7, 6, 5]; P

['Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

In [40]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h2_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_2)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.001653909683227539. Longitud solución: 12


['Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

In [41]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h3_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_2)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.004769802093505859. Longitud solución: 12


['Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

#### Estado inicial 3:

In [42]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h1_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_3)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.8963956832885742. Longitud solución: 18


['Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo']

In [43]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h2_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_3)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.03192949295043945. Longitud solución: 18


['Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo']

In [44]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h3_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_3)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.010374307632446289. Longitud solución: 18


['Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo']

#### Estado inicial 4:

In [45]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h1_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_4)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 317.86151003837585. Longitud solución: 25


['Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

In [46]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h2_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_4)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 3.134242296218872. Longitud solución: 25


['Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha']

In [47]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h3_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_4)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.21011114120483398. Longitud solución: 33


['Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hue

## Resumen resultados

| Estado Incial | Tiempo H1 | Longitud H1 | Tiempo H2 | Longitud H2 | Tiempo H3 | Longitud H3 |
| --- | --- | --- | --- | --- | --- | --- |
| (2, 8, 3, 1, 6, 4, 7, 0, 5) | 0.00025 | 5 | 0.0004 | 5 | 0.0006 | 5 |
| (4, 8, 1, 3, 0, 2, 7, 6, 5) | 0.007 | 12 | 0.0015 | 12 | 0.003 | 12 |
| (2, 1, 6, 4, 0, 8, 7, 5, 3) | 0.84 | 18 | 0.023 | 18 | 0.0049 | 18 |
| (5, 2, 3, 0, 4, 8, 7, 6, 1) | 265.54 | 25 | 2.605 | 25 | 0.1374 | 33 |

## Anexo: 8-puzzle representación alternativa de estado

A continuación se repite la implementación del problema del 8-puzzle usando una lista de listas (dos dimensiones) para representar el estado.
- Matriz: [[1, 2, 3], [8, 0, 4], [7, 6, 5]]

Usaremos dos funciones auxiliares para obtener la fila y la columna en la que se encuentra el hueco.

In [48]:
test =  [[1, 2, 3], [8, 4, 0], [7, 6, 5]]

In [49]:
def get_fila_hueco(estado):
    for i, fila in enumerate(estado):
        if 0 in fila:
            return i    

In [50]:
get_fila_hueco(test)

1

In [51]:
def get_columna_hueco(estado, fila_hueco=None):
    if not fila_hueco:
        fila_hueco = get_fila_hueco(estado)
    return estado[fila_hueco].index(0)

In [52]:
get_columna_hueco(test)

2

In [53]:
get_columna_hueco(test, 1)

2

A continuación definimos las acciones.

In [54]:
class MoverArriba(probee.Accion):
    def __init__(self):
        super().__init__("Mover hueco hacia arriba")
    
    def es_aplicable(self, estado):
        return get_fila_hueco(estado) > 0
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        fila_hueco = get_fila_hueco(estado)
        col_hueco = get_columna_hueco(estado, fila_hueco)
        nueva_fila_hueco = fila_hueco - 1
        
        nuevo_estado[fila_hueco][col_hueco] = estado[nueva_fila_hueco][col_hueco]
        nuevo_estado[nueva_fila_hueco][col_hueco] = 0
        return nuevo_estado 

    
class MoverAbajo(probee.Accion):
    def __init__(self):
        super().__init__("Mover hueco hacia abajo")
    
    def es_aplicable(self, estado):
        return get_fila_hueco(estado) < 2
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        fila_hueco = get_fila_hueco(estado)
        col_hueco = get_columna_hueco(estado, fila_hueco)
        nueva_fila_hueco = fila_hueco + 1
        
        nuevo_estado[fila_hueco][col_hueco] = estado[nueva_fila_hueco][col_hueco]
        nuevo_estado[nueva_fila_hueco][col_hueco] = 0
        return nuevo_estado 


class MoverDerecha(probee.Accion):
    def __init__(self):
        super().__init__("Mover hueco hacia la derecha")
    
    def es_aplicable(self, estado):
        return get_columna_hueco(estado) < 2
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        fila_hueco = get_fila_hueco(estado)
        col_hueco = get_columna_hueco(estado, fila_hueco)
        nueva_col_hueco = col_hueco + 1
        
        nuevo_estado[fila_hueco][col_hueco] = estado[fila_hueco][nueva_col_hueco]
        nuevo_estado[fila_hueco][nueva_col_hueco] = 0
        return nuevo_estado 
    

class MoverIzquierda(probee.Accion):
    def __init__(self):
        super().__init__("Mover hueco hacia la izquierda")
    
    def es_aplicable(self, estado):
        return get_columna_hueco(estado) > 0
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        fila_hueco = get_fila_hueco(estado)
        col_hueco = get_columna_hueco(estado, fila_hueco)
        nueva_col_hueco = col_hueco - 1
        
        nuevo_estado[fila_hueco][col_hueco] = estado[fila_hueco][nueva_col_hueco]
        nuevo_estado[fila_hueco][nueva_col_hueco] = 0
        return nuevo_estado 

In [55]:
from time import time

acciones = [MoverArriba(), MoverAbajo(), MoverDerecha(), MoverIzquierda()]
estado_final = [[1, 2, 3], [8, 0, 4], [7, 6, 5]]
estado_inicial_1 = [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
estado_inicial_2 = [[4, 8, 1], [3, 0, 2], [7, 6, 5]]
estado_inicial_3 = [[2, 1, 6], [4, 0, 8], [7, 5, 3]]
estado_inicial_4 = [[5, 2, 3], [0, 4, 8], [7, 6, 1]]
puzzle_8_1 = probee.ProblemaEspacioEstados(acciones, estado_inicial_1, [estado_final])
puzzle_8_2 = probee.ProblemaEspacioEstados(acciones, estado_inicial_2, [estado_final])
puzzle_8_3 = probee.ProblemaEspacioEstados(acciones, estado_inicial_3, [estado_final])
puzzle_8_4 = probee.ProblemaEspacioEstados(acciones, estado_inicial_4, [estado_final])

¿Necesitamos una heurística?

In [56]:
ti = time()
b_anchura = busqee.BusquedaEnAnchura(detallado=True)
sol1 = b_anchura.buscar(puzzle_8_1)
print('Tiempo transcurrido: {} segundos. Tamaño solución: {}'.format(round(time() - ti,2), len(sol1)))
sol1

Nodo: Estado: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]; Prof: 0
  Nodo: Estado: [[2, 8, 3], [1, 0, 4], [7, 6, 5]]; Prof: 1
  Nodo: Estado: [[2, 8, 3], [1, 6, 4], [7, 5, 0]]; Prof: 1
  Nodo: Estado: [[2, 8, 3], [1, 6, 4], [0, 7, 5]]; Prof: 1
    Nodo: Estado: [[2, 0, 3], [1, 8, 4], [7, 6, 5]]; Prof: 2
    Nodo: Estado: [[2, 8, 3], [1, 4, 0], [7, 6, 5]]; Prof: 2
    Nodo: Estado: [[2, 8, 3], [0, 1, 4], [7, 6, 5]]; Prof: 2
    Nodo: Estado: [[2, 8, 3], [1, 6, 0], [7, 5, 4]]; Prof: 2
    Nodo: Estado: [[2, 8, 3], [0, 6, 4], [1, 7, 5]]; Prof: 2
      Nodo: Estado: [[2, 3, 0], [1, 8, 4], [7, 6, 5]]; Prof: 3
      Nodo: Estado: [[0, 2, 3], [1, 8, 4], [7, 6, 5]]; Prof: 3
      Nodo: Estado: [[2, 8, 0], [1, 4, 3], [7, 6, 5]]; Prof: 3
      Nodo: Estado: [[2, 8, 3], [1, 4, 5], [7, 6, 0]]; Prof: 3
      Nodo: Estado: [[0, 8, 3], [2, 1, 4], [7, 6, 5]]; Prof: 3
      Nodo: Estado: [[2, 8, 3], [7, 1, 4], [0, 6, 5]]; Prof: 3
      Nodo: Estado: [[2, 8, 0], [1, 6, 3], [7, 5, 4]]; Prof: 3
      Nodo: Estado: 

['Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

In [57]:
# ti = time()
# b_profundidad = busqee.BusquedaEnProfundidad(detallado=False)
# sol = b_profundidad.buscar(puzzle_8_1)
# print('Tiempo transcurrido: {} segundos. Tamaño solución: {}'.format(round(time() - ti,2), len(sol)))
# sol

In [58]:
ti = time()
b_anchura = busqee.BusquedaEnAnchura(detallado=True)
sol2 = b_anchura.buscar(puzzle_8_2)
print('Tiempo transcurrido: {} segundos. Tamaño solución: {}'.format(round(time() - ti,2), len(sol2)))
sol2

Nodo: Estado: [[4, 8, 1], [3, 0, 2], [7, 6, 5]]; Prof: 0
  Nodo: Estado: [[4, 0, 1], [3, 8, 2], [7, 6, 5]]; Prof: 1
  Nodo: Estado: [[4, 8, 1], [3, 6, 2], [7, 0, 5]]; Prof: 1
  Nodo: Estado: [[4, 8, 1], [3, 2, 0], [7, 6, 5]]; Prof: 1
  Nodo: Estado: [[4, 8, 1], [0, 3, 2], [7, 6, 5]]; Prof: 1
    Nodo: Estado: [[4, 1, 0], [3, 8, 2], [7, 6, 5]]; Prof: 2
    Nodo: Estado: [[0, 4, 1], [3, 8, 2], [7, 6, 5]]; Prof: 2
    Nodo: Estado: [[4, 8, 1], [3, 6, 2], [7, 5, 0]]; Prof: 2
    Nodo: Estado: [[4, 8, 1], [3, 6, 2], [0, 7, 5]]; Prof: 2
    Nodo: Estado: [[4, 8, 0], [3, 2, 1], [7, 6, 5]]; Prof: 2
    Nodo: Estado: [[4, 8, 1], [3, 2, 5], [7, 6, 0]]; Prof: 2
    Nodo: Estado: [[0, 8, 1], [4, 3, 2], [7, 6, 5]]; Prof: 2
    Nodo: Estado: [[4, 8, 1], [7, 3, 2], [0, 6, 5]]; Prof: 2
      Nodo: Estado: [[4, 1, 2], [3, 8, 0], [7, 6, 5]]; Prof: 3
      Nodo: Estado: [[3, 4, 1], [0, 8, 2], [7, 6, 5]]; Prof: 3
      Nodo: Estado: [[4, 8, 1], [3, 6, 0], [7, 5, 2]]; Prof: 3
      Nodo: Estado: [[4, 8, 1]

['Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

In [59]:
ti = time()
b_anchura = busqee.BusquedaEnAnchura(detallado=False)
sol3 = b_anchura.buscar(puzzle_8_3)
print('Tiempo transcurrido: {} segundos. Tamaño solución: {}'.format(round(time() - ti,2), len(sol3)))
sol3

Tiempo transcurrido: 198.75 segundos. Tamaño solución: 18


['Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

Parece que si. En problemas como el 8-puzzle el estado inicial es muy importante. Vamos a probar diferentes heurísticas.

**Heurística:** número de piezas descolocadas respecto de su posición en el estado final.

In [60]:
def h1_ocho_puzzle(estado):
    # TO-DO
    cont=0
    estado_final = [[1, 2, 3], [8, 0, 4], [7, 6, 5]]
    for i in range(3):
        for j in range(3):
            if estado[i][j] != 0 and estado[i][j] != estado_final[i][j]: 
                cont += 1
    return cont

**Heurística:** suma de las distancias Manhattan de cada pieza a donde debería estar en el estado final.

In [61]:
def h2_ocho_puzzle(estado):
    # TO-DO
    posiciones_final = ((1,1),(0,0),(0,1),(0,2),(1,2),(2,2),(2,1),(2,0),(1,0)) 
    # posiciones_final[i] es el par de coordenadas (fila, columna) de la posición de i en el estado final  
    suma = 0                                                            
    for i in range(3):
        for j in range(3):
            valor = estado[i][j]  # valor que está en las coordenadas (i, j) de estado
            if valor != 0: # Excluimos el valor cero o 'hueco'.
                final_i, final_j = posiciones_final[valor] 
    # (final_i, final_j) es la posición donde debería estar el valor que está en la posición (i, j) de estado
                suma += abs(i - final_i) + abs(j - final_j) # distancia Manhattan
    return suma

**Heurística no admisible:** La siguiente heurística h3_ocho_puzzle se obtiene sumando a la heurística h2_ocho_puzzle una componente que cuantifica la "secuencialidad" en las casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en A*, con cada uno de los estados anteriores. Comentar los resultados.

In [62]:
def h3_ocho_puzzle(estado):
    # Para cada indice o posición de nuestro estado, este diccionario indica cuál es el siguiente 
    # indice a consultar para seguir el recorrido de las agujas del reloj en el cuadrado.
    suc_ocho_puzzle ={0: 1, 1: 2, 2: 5, 3: 0, 4: 4, 5: 8, 6: 3, 7: 6, 8: 7}  
    suc_ocho_puzzle ={(0, 0): (0, 1), (0, 1): (0, 2), (0, 2): (1, 2), (1, 0): (0, 0), (1, 1): (1, 1), 
                      (1, 2): (2, 2), (2, 0): (1, 0), (2, 1): (2, 0), (2, 2): (2, 1)} 
    
    # Comprueba si hay secuencialidad entre el valor en posición la posición i y el valor en 
    # la siguiente posición (segun las agujas del reloj). Si no hay secuencialidad se 
    # devuelve una penalización de dos unidades.
    def secuencialidad_aux(estado, coords):
        val=estado[coords[0]][coords[1]]
        if val == 0:
            return 0
        elif coords == (1, 1):
            return 1
        else:
            coords_suc=suc_ocho_puzzle[coords]
            val_sig = (val+1 if val<8 else 1)
            return 0 if val_sig == estado[coords_suc[0]][coords_suc[1]] else 2
    
    # Se comprueba la secuencilidad para todas las posiciones del estado y se suman 
    # las penalizaciones. Finalmente, se emite como resultado una combinación entre la 
    # heurística 2 y las penalizaciones de secuencialidad.
    def secuencialidad(estado):
        res= 0 
        for i in range(3):
            for j in range(3):
                res += secuencialidad_aux(estado, (i, j))
        return res    

    return h2_ocho_puzzle(estado) + 3*secuencialidad(estado)

Veamos la valoración que dan las diferentes heurísticas a los 4 estados iniciales que hemos definido.

In [63]:
estados_iniciales = [estado_inicial_1, estado_inicial_2, estado_inicial_3, estado_inicial_4]
for e in estados_iniciales:
    print('Estado: {}. h1: {}. h2: {}. h3: {}.'.
          format(e, h1_ocho_puzzle(e), h2_ocho_puzzle(e), h3_ocho_puzzle(e)))

Estado: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]. h1: 4. h2: 5. h3: 32.
Estado: [[4, 8, 1], [3, 0, 2], [7, 6, 5]]. h1: 5. h2: 12. h3: 30.
Estado: [[2, 1, 6], [4, 0, 8], [7, 5, 3]]. h1: 7. h2: 12. h3: 60.
Estado: [[5, 2, 3], [0, 4, 8], [7, 6, 1]]. h1: 4. h2: 11. h3: 38.


**Valoración nodos:** Hemos visto que las implementaciones proporcionadas de los diferentes algoritmos de búsqueda, tienen un parámetro `detallado` que permite imprimir información sobre los nodos que el algoritmo va visitando. Para algunos de los algoritmos se proporciona el campo `Valoración` el cual tendrá una interpretación diferente según cada algoritmo. Recordemos, que algunos algoritmos usan una valoración de cada nodo para ordenar la frontera, siendo esta:
- Búsqueda óptima: `coste`
- Búsqueda primero el mejor: `heurística`
- Búsqueda A*: `coste + heurística`

#### Estado inicial 1:

In [64]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h1_ocho_puzzle, detallado=True)
sol = busqueda_aestrella.buscar(puzzle_8_1)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Nodo: Estado: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]; Prof: 0; Valoración: 4; Coste: 0
  Nodo: Estado: [[2, 8, 3], [1, 0, 4], [7, 6, 5]]; Prof: 1; Valoración: 4; Coste: 1
    Nodo: Estado: [[2, 0, 3], [1, 8, 4], [7, 6, 5]]; Prof: 2; Valoración: 5; Coste: 2
    Nodo: Estado: [[2, 8, 3], [0, 1, 4], [7, 6, 5]]; Prof: 2; Valoración: 5; Coste: 2
      Nodo: Estado: [[0, 2, 3], [1, 8, 4], [7, 6, 5]]; Prof: 3; Valoración: 5; Coste: 3
        Nodo: Estado: [[1, 2, 3], [0, 8, 4], [7, 6, 5]]; Prof: 4; Valoración: 5; Coste: 4
          Nodo: Estado: [[1, 2, 3], [8, 0, 4], [7, 6, 5]]; Prof: 5; Valoración: 5; Coste: 5
Tiempo transcurrido: 0.0010027885437011719. Longitud solución: 5


['Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

In [65]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h2_ocho_puzzle, detallado=True)
sol = busqueda_aestrella.buscar(puzzle_8_1)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Nodo: Estado: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]; Prof: 0; Valoración: 5; Coste: 0
  Nodo: Estado: [[2, 8, 3], [1, 0, 4], [7, 6, 5]]; Prof: 1; Valoración: 5; Coste: 1
    Nodo: Estado: [[2, 0, 3], [1, 8, 4], [7, 6, 5]]; Prof: 2; Valoración: 5; Coste: 2
      Nodo: Estado: [[0, 2, 3], [1, 8, 4], [7, 6, 5]]; Prof: 3; Valoración: 5; Coste: 3
        Nodo: Estado: [[1, 2, 3], [0, 8, 4], [7, 6, 5]]; Prof: 4; Valoración: 5; Coste: 4
          Nodo: Estado: [[1, 2, 3], [8, 0, 4], [7, 6, 5]]; Prof: 5; Valoración: 5; Coste: 5
Tiempo transcurrido: 0.0019614696502685547. Longitud solución: 5


['Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

In [66]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h3_ocho_puzzle, detallado=True)
sol = busqueda_aestrella.buscar(puzzle_8_1)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Nodo: Estado: [[2, 8, 3], [1, 6, 4], [7, 0, 5]]; Prof: 0; Valoración: 32; Coste: 0
  Nodo: Estado: [[2, 8, 3], [1, 0, 4], [7, 6, 5]]; Prof: 1; Valoración: 23; Coste: 1
    Nodo: Estado: [[2, 0, 3], [1, 8, 4], [7, 6, 5]]; Prof: 2; Valoración: 20; Coste: 2
      Nodo: Estado: [[0, 2, 3], [1, 8, 4], [7, 6, 5]]; Prof: 3; Valoración: 20; Coste: 3
        Nodo: Estado: [[1, 2, 3], [0, 8, 4], [7, 6, 5]]; Prof: 4; Valoración: 14; Coste: 4
          Nodo: Estado: [[1, 2, 3], [8, 0, 4], [7, 6, 5]]; Prof: 5; Valoración: 5; Coste: 5
Tiempo transcurrido: 0.0017957687377929688. Longitud solución: 5


['Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

#### Estado inicial 2:

In [67]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h1_ocho_puzzle, detallado=True)
sol = busqueda_aestrella.buscar(puzzle_8_2)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Nodo: Estado: [[4, 8, 1], [3, 0, 2], [7, 6, 5]]; Prof: 0; Valoración: 5; Coste: 0
  Nodo: Estado: [[4, 0, 1], [3, 8, 2], [7, 6, 5]]; Prof: 1; Valoración: 6; Coste: 1
  Nodo: Estado: [[4, 8, 1], [3, 2, 0], [7, 6, 5]]; Prof: 1; Valoración: 6; Coste: 1
  Nodo: Estado: [[4, 8, 1], [0, 3, 2], [7, 6, 5]]; Prof: 1; Valoración: 6; Coste: 1
  Nodo: Estado: [[4, 8, 1], [3, 6, 2], [7, 0, 5]]; Prof: 1; Valoración: 7; Coste: 1
    Nodo: Estado: [[4, 1, 0], [3, 8, 2], [7, 6, 5]]; Prof: 2; Valoración: 7; Coste: 2
    Nodo: Estado: [[0, 4, 1], [3, 8, 2], [7, 6, 5]]; Prof: 2; Valoración: 7; Coste: 2
    Nodo: Estado: [[4, 8, 0], [3, 2, 1], [7, 6, 5]]; Prof: 2; Valoración: 7; Coste: 2
    Nodo: Estado: [[0, 8, 1], [4, 3, 2], [7, 6, 5]]; Prof: 2; Valoración: 7; Coste: 2
    Nodo: Estado: [[4, 8, 1], [3, 2, 5], [7, 6, 0]]; Prof: 2; Valoración: 8; Coste: 2
    Nodo: Estado: [[4, 8, 1], [7, 3, 2], [0, 6, 5]]; Prof: 2; Valoración: 8; Coste: 2
      Nodo: Estado: [[4, 1, 2], [3, 8, 0], [7, 6, 5]]; Prof: 3; Va

['Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

In [68]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h2_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_2)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.0030307769775390625. Longitud solución: 12


['Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

In [69]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h3_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_2)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.005568742752075195. Longitud solución: 12


['Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda']

#### Estado inicial 3:

In [70]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h1_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_3)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 1.271225929260254. Longitud solución: 18


['Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo']

In [71]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h2_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_3)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.04490399360656738. Longitud solución: 18


['Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo']

In [72]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h3_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_3)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.0047512054443359375. Longitud solución: 18


['Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo']

#### Estado inicial 4:

In [73]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h1_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_4)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 513.4268336296082. Longitud solución: 25


['Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']

In [74]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h2_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_4)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 3.8561441898345947. Longitud solución: 25


['Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha']

In [75]:
ti = time()
busqueda_aestrella = busqee.BusquedaAEstrella(h3_ocho_puzzle, detallado=False)
sol = busqueda_aestrella.buscar(puzzle_8_4)
print('Tiempo transcurrido: {}. Longitud solución: {}'.format(time() - ti, len(sol)))
sol

Tiempo transcurrido: 0.16264128684997559. Longitud solución: 29


['Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia arriba',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia la derecha',
 'Mover hueco hacia arriba',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia la izquierda',
 'Mover hueco hacia abajo',
 'Mover hueco hacia la derecha']